In [1]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
text4000 = pd.read_csv(".//Data/4000/4000-stories-VAD.csv")

In [3]:
text4000.head(1)

,url,length,title,text_no,author,story,valence,arousal,dominance,text_id,...,290,291,292,293,294,295,296,297,298,299
0,https://americanliterature.com/author/eleanor-...,15044,"Peace on Earth, Good-Will to Dogs",0,Eleanor Hallowell Abbott,"PART I\n\nIf you don't like Christmas stories,...",0.592896,0.397839,0.569567,0_eleanor-hallowell-abbott-peace-on-earth-good...,...,3.793141,3.837345,3.778354,-0.815515,-0.72044,-10.738245,-8.765683,0.875089,-10.176691,1.736791


In [4]:
text4000.keys()

Index(['url', 'length', 'title', 'text_no', 'author', 'story', 'valence',
       'arousal', 'dominance', 'text_id',
       ...
       '290', '291', '292', '293', '294', '295', '296', '297', '298', '299'],
      dtype='object', length=317)

In [5]:
texts= list(text4000.story.values)
texts[2]

'His affair that night was prosy. He was intending the murder of an old Spanish woman around the corner, on the Bowery, whom he had known for years, with whom he had always exchanged courteous greetings, and whom he neither liked nor disliked.\n\nHe did kill her; and she knew that he was going to the minute he came into her stuffy, smelly shop, looming tall and bland, and yellow, and unearthly Chinese from behind the shapeless bundles of second-hand goods that cluttered the doorway. He wished her good evening in tones that were silvery, but seemed tainted by something unnatural. She was uncertain what it was, and this very uncertainty increased her horror. She felt her hair rise as if drawn by a shivery wind.\n\nAt the very last she caught a glimmer of the truth in his narrow-lidded, purple-black eyes. But it was too late.\n\nThe lean, curved knife was in his hand and across her scraggy throat—there was a choked gurgle, a crimson line broadening to a crimson smear, a thudding fall—and 

In [6]:
text_limiter=10000 #10000 zeichen
joined_text = " ".join(texts)
limited_text = joined_text[:text_limiter]
limited_text

'PART I\n\nIf you don\'t like Christmas stories, don\'t read this one!\n\nAnd if you don\'t like dogs I don\'t know just what to advise you to do!\n\nFor I warn you perfectly frankly that I am distinctly pro-dog and distinctly pro-Christmas, and would like to bring to this little story whatever whiff of fir-balsam I can cajole from the make-believe forest in my typewriter, and every glitter of tinsel, smudge of toy candle, crackle of wrapping paper, that my particular brand of brain and ink can conjure up on a single keyboard! And very large-sized dogs shall romp through every page! And the mercury shiver perpetually in the vicinity of zero! And every foot of earth be crusty-brown and bare with no white snow at all till the very last moment when you\'d just about given up hope! And all the heart of the story is very,—oh very young!\n\nFor purposes of propriety and general historical authenticity there are of course parents in the story. And one or two other oldish persons. But they all

In [10]:
# Tokenizer erstellen und Texte darauf anwenden
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(limited_text.lower())

In [12]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [15]:
context_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - context_words):
    input_words.append(tokens[i:i + context_words])
    next_words.append(tokens[i + context_words])

In [39]:
X = np.zeros((len(input_words), context_words, len(unique_tokens)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [40]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i,j, unique_token_index[word]] = 1
    Y[i, unique_token_index[next_words[i]]] = 1

In [48]:
model = Sequential()
model.add(LSTM(128, input_shape=(context_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [50]:
model.compile(loss="categorical_crossentropy", optimizer = RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(X, Y, batch_size=128, epochs=10, shuffle=True)

Epoch 1/10
15/15 [==============================] - 4s 45ms/step - loss: 6.2435 - accuracy: 0.0261
Epoch 2/10
15/15 [==============================] - 1s 45ms/step - loss: 5.9110 - accuracy: 0.0367
Epoch 3/10
15/15 [==============================] - 1s 46ms/step - loss: 5.8788 - accuracy: 0.0417
Epoch 4/10
15/15 [==============================] - 1s 45ms/step - loss: 5.8452 - accuracy: 0.0356
Epoch 5/10
15/15 [==============================] - 1s 44ms/step - loss: 5.8249 - accuracy: 0.0417
Epoch 6/10
15/15 [==============================] - 1s 45ms/step - loss: 5.7985 - accuracy: 0.0422
Epoch 7/10
15/15 [==============================] - 1s 46ms/step - loss: 5.7741 - accuracy: 0.0384
Epoch 8/10
15/15 [==============================] - 1s 46ms/step - loss: 5.7151 - accuracy: 0.0395
Epoch 9/10
15/15 [==============================] - 1s 45ms/step - loss: 5.6536 - accuracy: 0.0461
Epoch 10/10
15/15 [==============================] - 1s 46ms/step - loss: 5.4783 - accuracy: 0.0528


In [51]:
model.save("myFirstTestModel.h5")

In [52]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, context_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
                             
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [59]:
possible = predict_next_word("His that night was",5)

1/1 [==============================] - 0s 18ms/step


In [62]:
print([unique_tokens[idx] for idx in possible])

['of', 's', 'it', 'the', 'and']


In [79]:
with open('MetaData/context_words.pickle', 'wb') as file:
    pickle.dump(context_words, file)
with open('MetaData/unique_tokens.pickle', 'wb') as file:
    pickle.dump(unique_tokens, file)
with open('MetaData/unique_token_index.pickle', 'wb') as file:
    pickle.dump(unique_token_index, file)   